### Import files

In [ ]:
import numpy as np
import pandas as pd
import os
import statistics
import heapq
import re, string

path = "C:\\Users\pault\OneDrive\Documenten\GitHub\input" # pauls path
#path = "/home/afalbrecht/Documents/Leren en Beslissen/" #add your path here, dit is een test
os.chdir(path)

train = pd.read_csv('train.tsv', delimiter='\t', encoding='utf-8')
print("train size:", train.size)

test = pd.read_csv('test.tsv', delimiter='\t', encoding='utf-8')
print("test size:", test.size)

### Small test space

In [ ]:
print(train.size)

train.get_value(1, "price")
a = statistics.mean([1, 4, 5, 6, 4, 3, 2])
print(a)

train.head(100)

### Functions

In [ ]:
sample = train.loc[0:10]

# sorts a column and returns a dictionary with a list of indexes that have the same text/number in that column. 
def sort_index_by_column(data, column):
    dct = {'nan': []}
    for index, row in data.iterrows():
        if row[column] == row[column]:
            if row[column] in dct: 
                dct[row[column]].append(index)
            else: 
                dct[row[column]] = [index]
        else:
            dct["nan"].append(index)
    return dct

#deletes all the keys that have a list with only one value 
def delete_einzelgangers(dictionary):
    new_dict = {}
    for key in dictionary:
        if len(dictionary[key]) != 1:
               new_dict[key] = dictionary[key]
    return new_dict

# for every list, replace all the indexes with prices
def replace_index_with_price(dicti, data):
    new_dict = {}
    for key in dicti: 
        lst = []
        for ind in dicti[key]: 
            lst.append(data.get_value(ind, "price"))
        new_dict[key] = lst
    return new_dict

# calculates various things for every key
def mean_var_amount_per_dict_key(dicti):
    mean_dict = {}
    for key in dicti: 
        mean = statistics.mean(dicti[key])
        variance = statistics.variance(dicti[key])
        amount = len(dicti[key])
        mean_dict[key] = [mean, variance, int(amount)]
    return mean_dict

# gets the amount of times a word is being used in the item description
def get_common_words(data):
    dct = {}
    for index, row in data.iterrows():
        for word in row["item_description"].split(" "):
            if word in dct: 
                dct[word] += 1
            else: 
                dct[word] = 1
    highest_keys = sorted(dct, key=dct.get, reverse=True)
    new_dct = {}
    for key in highest_keys:
        new_dct[key] = dct[key]
    return new_dct

def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

# every item (except a few NaN's) have three categories. Here we split those in three different columns.
def split_categories(data):
    data["first_cat"] = ""
    data["second_cat"] = ""
    data["third_cat"] = ""
    for index, row in data.iterrows():
        if row["category_name"] == row["category_name"]:
            data.set_value(index, "first_cat", row["category_name"].split("/")[0])
            data.set_value(index, "second_cat", row["category_name"].split("/")[1])
            data.set_value(index, "third_cat", row["category_name"].split("/")[2])
        #print(row)
    return data

# replaces the item description with 
def change_item_description(data):
    for index, row in data.iterrows():
        if isinstance(row["item_description"], list) == True:
            break
        else:
            des = row["item_description"]
            des = des.lower()
            regex = re.compile('[' +re.escape(string.punctuation) + '\\r\\t\\n]')
            txt = regex.sub(" ", des)
            stopwords = """rm i me my myself we our ours ourselves you your yours yourself yourselves he him 
            his himself she her hers herself it its itself they them their theirs themselves what which who whom 
            this that these those am is are was were be been being have has had having do does did doing a an the 
            and but if or because as until while of at by for with about against between into through during before 
            after above below to from up down in out on off over under again further then once here there when where 
            why how all any both each few more most other some such no nor not only own same so than too very s t can 
            will just don should now"""
            words = [w for w in txt.split(" ") \
                     if not w in stopwords]
            data.set_value(index, "item_description", words)        

### Description Parser

In [ ]:
change_item_description(sample)
split_categories(sample)

sample.head(5)

def lists_for_vector(data):
    cat = []
    item_des = []
    
    for index, row in data.iterrows():
        #print(row["first_cat"])
        cat.append(row["first_cat"])
        cat.append(row["second_cat"])
        cat.append(row["third_cat"])
        for word in row["item_description"]:
            item_des.append(word)
    item_des = list(set(item_des))
    cat = list(set(cat))
    return item_des, cat

change_item_description(train)
split_categories(train)
d, c = lists_for_vector(train)

print(len(d))
print(len(c))

train.head(5)

### Print stuff

In [ ]:
sort_brand_name_ind = delete_einzelgangers(sort_index_by_column(sample, "brand_name"))
# for key in sort_brand_name_ind: 
#     print(key + ":", sort_brand_name_ind[key])  

sort_brand_name_price = replace_index_with_price(sort_brand_name_ind, sample)
# for key in sort_brand_name_price: 
#     print(key + ":", sort_brand_name_price[key])
    
sort_brand_name_mva = mean_var_amount_per_dict_key(sort_brand_name_price)
# for key in sort_brand_name_mva:
#     print(key + " - mean:", int(sort_brand_name_mva[key][0]), "variance:", 
#     int(sort_brand_name_mva[key][1]), "amount:", sort_brand_name_mva[key][2])
        
words = get_common_words(sample)
#for word in words: 
#    print(word + ":", words[word])

# for index, row, in sample.iterrows():
#     descr = row["item_description"]
#     if hasNumbers(descr) == True: 
#         print(row["name"], descr, "\n")



In [ ]:
import numpy as np 
    
test_sen = np.array([['een', 'mooie', 'aap'], 
                     ['een', 'goede', 'aap', ''], 
                     ['een', 'vette', 'slang'], 
                     ['een', 'mooie', 'slang', ''], 
                     ['een', 'grote', 'slang'], 
                     ['een', 'gave', 'aap', ''], 
                     ['een', 'mooie', 'koe'], 
                     ['een', 'lieve', 'koe', ''],
                     ['een', 'gave', 'eend'], 
                     ['een', 'vette', 'eend', ''], 
                     ['een', 'mooie', 'eend', ''],
                     ['een', 'lieve', 'eend', ''],
                     ['een', 'leuke', 'kip', ''],
                     ['een', 'mooie', 'kip', ''], 
                     ['een', 'grote', 'kip', ''],
                     ['een', 'vette', 'kip'], 
                     ['een', 'mooie', 'kip', ''],
                     ['een', 'lieve', 'gans', ''], 
                     ['een', 'mooie', 'gans'], 
                     ['een', 'gave', 'gans'], 
                     ['een', 'leuke', 'gans'], 
                     ['een', 'leuke', 'aap', ''], 
                     ['een', 'slechte', 'aap', ''],
                     ['een', 'lelijke', 'aap', ''], 
                     ['een', 'kutte', 'aap', ''],
                     ['een', 'slechte', 'slang', ''], 
                     ['een', 'kutte', 'slang'], 
                     ['een', 'kut', 'slang', ''], 
                     ['een', 'stomme', 'koe', ''],
                     ['een', 'kutte', 'koe'],
                     ['een', 'fucking', 'koe', ''],
                     ['een', 'domme', 'koe', ''], 
                     ['een', 'slechte', 'koe'], 
                     ['een', 'stomme', 'eend', ''], 
                     ['een', 'slechte', 'eend', ''], 
                     ['een', 'kut', 'eend', ''], 
                     ['een', 'stomme', 'kip', ''], 
                     ['een', 'domme', 'kip', ''], 
                     ['een', 'stomme', 'kut'],
                     ['een', 'slechte', 'kip']])

print(len(test_sen))

test_goodorbad = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ]

print(len(test_goodorbad))


